# Python example for Climatic Research Unit (CRU) time-series (TS) data

## CRU TS data

The following guide will assist with the visualisation of the Climatic Research Unit (CRU) gridded time-series (TS) dataset. For more information on this data, or to understand the ways in which it can be downloaded, please see the CEDA CRU data user guide.

In [14]:
venv_dir = venvs_dir = os.path.join(os.path.expanduser("~"), "nb-venvs","venv-notebook")

In [15]:
# Activate the venv
activate_file = os.path.join(venv_dir, "bin", "activate_this.py")
exec(open(activate_file).read(), dict(__file__=activate_file))

In [24]:
import netCDF4
import numpy as np
import import_ipynb
import cru_data_manipulation

### Exporting and saving subsets of data

This section will explain some methods of exporting subsets of data you have created so you can plot that data or save it for something else.

##### 1. Saving data to a NetCDF file

We have previously calculated a global temperature anomaly subset, where the values in 2017 are compared to a reference period (1961-1990). For this subset, the temperature data is a function of latitude and longitude. We are going to save it in a NetCDF file.

Firstly, we have to create the netcdf file and ensure we can write to it. This is done using the code below.

In [20]:
ncfile =  netCDF4.Dataset('global_temp_anomaly_2017.nc', mode='w',format='NETCDF4_CLASSIC')

Next we set up the dimensions. The code below gives each a name and a size. The CRU data has a high resolution (0.5x0.5 degree) and so these values are double what is normally expected for the globe (180 latitude values and 360 longitude). This can be identified by printing the length of the latitude and longitude variables set originally.

In [21]:
lat_dim = ncfile.createDimension('lat', 360)
lon_dim = ncfile.createDimension('lon', 720)

We can add certain metadata to our netcdf file to make it more informative. This is useful if we pass the data to someone else or use it after a long time. The below code is just adding a title to the data. There are lots of other details that can be added.

In [22]:
ncfile.title = 'CRU TS v4.02 Global temperature anomaly for 2017 (wrt 1961-1990)'

Next we add variables. We need latitude, longitude and temperature. For each it is required we add a unit and standard name. When creating the variable we give it a name, datatype and shape. The shapes are specified using the dimension names. We can then print a variable to check it looks correct. 

In [25]:
lat_var = ncfile.createVariable('lat',np.float32,('lat',))
lat_var.units = 'degrees_north'
lat_var.standard_name = 'latitude'

lon_var = ncfile.createVariable('lon', np.float32, ('lon',))
lon_var.units = 'degrees_east'
lon_var.standard_name = 'longitude'

temp_var = ncfile.createVariable('tmp', np.float, ('lat','lon'))
temp_var.units = 'K'
temp_var.standard_name = 'air_temperature'
print(temp_var)

<class 'netCDF4._netCDF4.Variable'>
float64 tmp(lat, lon)
    units: K
    standard_name: air_temperature
unlimited dimensions: 
current shape = (360, 720)
filling on, default _FillValue of 9.969209968386869e+36 used



Lastly, we assign data to each variable and close the netcdf file.

In [26]:
lat_var[:] = cru_data_manipulation.lat
lon_var[:] = cru_data_manipulation.lon
temp_var[:,:] = cru_data_manipulation.temp_2017_anom

In [27]:
ncfile.close()

The Linux command below is printing the metadata from our new netcdf file to make sure it has all gone in as expected.

In [28]:
!ncdump -h global_temp_anomaly_2017.nc

netcdf global_temp_anomaly_2017 {
dimensions:
	lat = 360 ;
	lon = 720 ;
variables:
	float lat(lat) ;
		lat:units = "degrees_north" ;
		lat:standard_name = "latitude" ;
	float lon(lon) ;
		lon:units = "degrees_east" ;
		lon:standard_name = "longitude" ;
	double tmp(lat, lon) ;
		tmp:units = "K" ;
		tmp:standard_name = "air_temperature" ;

// global attributes:
		:title = "CRU TS v4.02 Global temperature anomaly for 2017 (wrt 1961-1990)" ;
}


##### 2. Exporting a spatial subset

Now we are going to export the spatial subset we created earlier. This was precipitation data subset specifically for the European region and sliced for 1 month. 

In [31]:
import csv

with open('cru_pre_eu_01-1901.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerows(cru_data_manipulation.rain_eu)

##### 3. Exporting a temporal subset

Now we are going to export the temporal subset subset we created earlier. This was precipitation data subset specifically for 2017 (Jan-Dec) for the whole globe. An average was calculated.

The code below is the same as before - to save data to a csv file.

In [30]:
with open('cru_pre_2017_average.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerows(cru_data_manipulation.rain_2017)